In [ ]:
#!/usr/bin/env python
# coding: utf-8

import gridfs
import pymongo
import bson

import datetime

start = datetime.datetime(2019, 7, 23, 0, 0, 0)
end = datetime.datetime(2019, 7, 26, 0, 0, 0)

MONGO_CLIENT = pymongo.MongoClient('mongodb://rccc-physicssvr:27017/')
DB = MONGO_CLIENT['tqms']
GRID_FS = gridfs.GridFS(DB, collection='FileManagerFS')

doses_in_date_range = list(DB.FileManagerFS.files.find({
    'filename': bson.regex.Regex(r'PerBeamDose.*\.dcm'),
    'uploadDate': {'$lt': end, '$gte': start}
}))

for result in doses_in_date_range:
    _id = result['_id']
    dicom_file = GRID_FS.find_one({'_id': _id})
    save_filename = f'{str(dicom_file._id)}_{dicom_file.filename}'

    with open(save_filename, 'wb') as a_file:
        a_file.write(dicom_file.read())
